# 思路

- 将同一篇文章split为2段，相似度为1，不同的文章的相似度为0
- jieba分词，将用词向量模型，得到每个词的词向量。
- 将词向量输入到rnn，训练相似度模型

## 1、数据处理
- 数据地址：链接:https://pan.baidu.com/s/1kQsaRgpEAMDib1LRa2QHqw  密码:bq52


In [3]:
# 1、处理数据
import os
import copy
from collections import defaultdict
import jieba


data_dict = defaultdict(list)
single_data_dict = defaultdict(list)
path = 'data/train'
root_dirs = os.listdir(path)


for dir_ in root_dirs:
    fir_dir = os.path.join(path, dir_)
    for root, sec_dir, files in os.walk(fir_dir):
        for file in files:
            file_abs_path = os.path.join(root, file)
            try:
                data = open(file_abs_path, 'r', encoding='gbk')
                for line in data:
                    line = line.replace('$LOTOzf$', '')
                    if len(line) < 80:
                        single_data_dict[dir_].append(line)
                        continue
                    fir_sent = line[:len(line) // 2]
                    sec_sent = line[len(line) // 2:]
                    data_dict[dir_].append(copy.deepcopy([fir_sent, sec_sent]))
            except:
                print(file_abs_path)
                continue          


data/train/体育/300.txt


In [5]:
import random

    
def cut_words(text):
    return list(jieba.cut(text))
    
    
classify_dict = ['文学', '体育', '女性', '校园']
writer = open('data/semantic_disambiguation_lstm.txt', 'w')
temp_dict = {}

for k, v in data_dict.items():
    temp = ['文学', '体育', '女性', '校园']
    cur_classify = classify_dict.index(k)
    temp.pop(cur_classify)
    other_data = [d for i, data in data_dict.items() if i != k for da in data for d in da]
    for sents in v:
        fir_sent = sents[0]
        sec_sent = sents[1]
        single_data_dict[k].extend([fir_sent, sec_sent])
        random_index = random.randint(0, len(single_data_dict[k])-1)
        
        random_index2 = random.randint(0,len(other_data)-1)
        neg_fir_sent = single_data_dict[k].pop(random_index)
        neg_sec_sent = other_data[random_index2]
        
        fir_sent = cut_words(fir_sent)
        sec_sent = cut_words(sec_sent)
        neg_fir_sent = cut_words(neg_fir_sent)
        neg_sec_sent = cut_words(neg_sec_sent)
        
        writer_str = f'{k}\t{fir_sent}\t{sec_sent}\t{1}\n'
        writer.write(writer_str)
        writer_str = f'{k}\t{neg_fir_sent}\t{neg_sec_sent}\t{0}\n'
        writer.write(writer_str)

writer.close()   
    

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/yuhe/Application/python36/lib/python3.6/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmp6ru9qwp9' -> '/tmp/jieba.cache'
Loading model cost 0.706 seconds.
Prefix dict has been built successfully.


## 中文词向量模型地址：
- 链接:https://pan.baidu.com/s/1vYXKfkNjOSIhUICa_mgl6A  密码:lv08

In [5]:
## 2、处理词向量,作词向量字典
import numpy as np
import torch 


vec_path = 'word_vector/sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5'
vector_dict = {}
with open(vec_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i == 0:
            continue
        line = line.split()
        vector_dict[line[0]] = np.array(line[1:])  

## 构造训练集

In [6]:
import random


data_set = open('data/semantic_disambiguation_lstm.txt', 'r').readlines()
random.shuffle(data_set)

data_array = []
for d in data_set:
    try:
        temp_data = d.split('\t')
        a = eval(temp_data[1])
        b = eval(temp_data[2])
        lable = int(temp_data[-1].strip('\n'))
        vec_a = np.concatenate([vector_dict.get(word).reshape(300, -1).astype(float) for word in a if vector_dict.get(word) is not None], axis=1)
        vec_b = np.concatenate([vector_dict.get(word).reshape(300, -1).astype(float) for word in b if vector_dict.get(word) is not None], axis=1)
    except:
        continue
    data_array.append([vec_a, vec_b, lable])

test_set = data_array[:len(data_array) // 7]
train_set = data_array[len(data_array) // 7:]


In [11]:
len(test_set[0])

3

In [34]:
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(123) #保证每次运行初始化的随机数相同

# vocab_size = 5000   #词表大小
embedding_size = 300   #词向量维度
num_classes = 2    #二分类
# sentence_max_len = 64  #单个句子的长度
hidden_size = 256

num_layers = 1  #一层lstm
num_directions = 2  #双向lstm
lr = 1e-3
batch_size = 1  
epochs = 6

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

#Bi-LSTM模型
class RNNModel(nn.Module):
    def __init__(self, embedding_size,hidden_size, num_layers, num_directions, num_classes):
        super(RNNModel, self).__init__()
        
        self.input_size = embedding_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_directions = num_directions
        
        
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers = num_layers, bidirectional = (num_directions == 2))
        self.attention_weights_layer = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(inplace=True)
        )
        self.liner = nn.Linear(hidden_size, num_classes)
        self.act_func = nn.Softmax(dim=1)
    
    def forward(self, x):
        #lstm的输入维度为 [seq_len, batch, input_size]
        #x [batch_size, sentence_length, embedding_size]
        x = x.permute(1, 0, 2)         #[sentence_length, batch_size, embedding_size]
        
        #由于数据集不一定是预先设置的batch_size的整数倍，所以用size(1)获取当前数据实际的batch
        batch_size = x.size(1)
        
        #设置lstm最初的前项输出
        h_0 = torch.randn(self.num_layers * self.num_directions, batch_size, self.hidden_size).to(device)
        c_0 = torch.randn(self.num_layers * self.num_directions, batch_size, self.hidden_size).to(device)
        
        #out[seq_len, batch, num_directions * hidden_size]。多层lstm，out只保存最后一层每个时间步t的输出h_t
        #h_n, c_n [num_layers * num_directions, batch, hidden_size]
        out, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        
        #将双向lstm的输出拆分为前向输出和后向输出
        (forward_out, backward_out) = torch.chunk(out, 2, dim = 2)
        out = forward_out + backward_out  #[seq_len, batch, hidden_size]
        out = out.permute(1, 0, 2)  #[batch, seq_len, hidden_size]
        
        #为了使用到lstm最后一个时间步时，每层lstm的表达，用h_n生成attention的权重
        h_n = h_n.permute(1, 0, 2)  #[batch, num_layers * num_directions,  hidden_size]
        h_n = torch.sum(h_n, dim=1) #[batch, 1,  hidden_size]
        h_n = h_n.squeeze(dim=1)  #[batch, hidden_size]
        
        attention_w = self.attention_weights_layer(h_n)  #[batch, hidden_size]
        attention_w = attention_w.unsqueeze(dim=1) #[batch, 1, hidden_size]
        # [batch, 1, hidden_size]  * [batch, seq_len, hidden_size]
        attention_context = torch.bmm(attention_w, out.transpose(1, 2))  
        softmax_w = F.softmax(attention_context, dim=-1)  #[batch, 1, seq_len],权重归一化
        x = torch.bmm(softmax_w, out)  #[batch, 1, hidden_size]
        x = x.squeeze(dim=1)  #[batch, hidden_size]
        x = self.liner(x)
        x = self.act_func(x)
        return x
     
        
def test(model, test_loader, loss_func):
    model.eval()
    loss_val = 0.0
    corrects = 0.0
    for datas in test_loader:
        inputs_0 = torch.from_numpy(datas[0]).float()
        inputs_1 = torch.from_numpy(datas[1]).float()
        if datas[2] == 1:
            labels = torch.LongTensor([0, 1]).unsqueeze(0).float()
        else:
            labels = torch.LongTensor([1, 0]).unsqueeze(0).float()
        inputs = torch.cat([inputs_0, inputs_1], dim=1).permute(1, 0).unsqueeze(0) # len x 300
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        preds = model(inputs)
        loss = loss_func(preds, labels)
        
        loss_val += loss.item() * inputs.size(0)
        
        #获取预测的最大概率出现的位置
        preds = torch.argmax(preds, dim=1)
        labels = torch.argmax(labels, dim=1)
        corrects += torch.sum(preds == labels).item()
    test_loss = loss_val / len(test_loader)
    test_acc = corrects / len(test_loader)
    print("Test Loss: {}, Test Acc: {}".format(test_loss, test_acc))
    return test_acc


def train(model, train_loader,test_loader, optimizer, loss_func, epochs, model_save_path):
    best_val_acc = 0.0
    best_model_params = copy.deepcopy(model.state_dict())
    train_loss = 0
    step = 0
    total_train_loss = []
    total_eval_loss = []
    mean_loss = []
    for epoch in range(epochs):
        model.train()
        loss_val = 0.0
        corrects = 0.0
        
        for datas in train_loader:
            inputs_0 = torch.from_numpy(datas[0]).float()
            inputs_1 = torch.from_numpy(datas[1]).float()
            if datas[2] == 1:
                labels = torch.LongTensor([0, 1]).unsqueeze(0).float()
            else:
                labels = torch.LongTensor([1, 0]).unsqueeze(0).float()
            inputs = torch.cat([inputs_0, inputs_1], dim=1).permute(1, 0).unsqueeze(0) # len x 300
            inputs = inputs.to(device)
            labels = labels.to(device)
            preds = model(inputs)
            loss = loss_func(preds, labels)
            step += 1
            train_loss += loss.mean().item()
            if step % 10 == 0:
                mean_loss.append(train_loss / step)
                print(f'Epoch {epoch}/ {step} train_loss  {train_loss / step}')
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_val += loss.item() * inputs.size(0)
            total_train_loss.append(loss.item())            
            
            #获取预测的最大概率出现的位置
            preds = torch.argmax(preds, dim=1)
            labels = torch.argmax(labels, dim=1)
            corrects += torch.sum(preds == labels).item()
        train_loss = loss_val / len(train_loader)
        train_acc = corrects / len(train_loader)
        
        if(epoch % 1 == 0):
            print("Train Loss: {}, Train Acc: {}".format(train_loss, train_acc))
            test_acc = test(model, test_loader, loss_func)
            if(best_val_acc < test_acc):
                torch.save(model.state_dict(), model_save_path)
                print(f'保存模型成功! 路径为: ’{model_save_path}‘')
                best_val_acc = test_acc
                best_model_params = copy.deepcopy(model.state_dict())
    model.load_state_dict(best_model_params)
    
    return model, total_train_loss, mean_loss


def predict(model, origin, entity_descs:list):
    """
    origin: 原始文本
    entity_descs:原始文本中具有歧义的实体词的多个异项的相关描述放到entity_descs中。
    比如：origin：苹果12马上就要发布了。
    歧义实体为”苹果“
    entity_descs：
    ['《苹果》是由李玉执导，范冰冰、佟大为、梁家辉、金燕玲领衔主演的黑色幽默剧情电影。于2007年5月18日在中国大陆上映。影片讲述了两个贫富家庭之间离奇诡异的矛盾冲突和感情错位的戏剧性故事。'
, 'iPhone 12是美国苹果公司研发的iPhone手机，采用了直面边框设计，支持5G，搭载A14 Bionic芯片，双镜头后置摄像头系统。支持北斗导航 [1]  ，有黑色、白色、红色、绿色、蓝色五种配色。'
]
    entity_descs中的句子分别和origin做相似度得分，得分高的为正确意项
    """
    scores = []
    for entity_desc in entity_descs:
        origin_vector = np.concatenate([vector_dict.get(word).reshape(300, -1).astype(float) for word in origin if vector_dict.get(word) is not None], axis=1)
        origin_vector = torch.from_numpy(origin_vector).float()
        entity_desc_vector = np.concatenate([vector_dict.get(word).reshape(300, -1).astype(float) for word in entity_desc if vector_dict.get(word) is not None], axis=1)
        entity_desc_vector = torch.from_numpy(entity_desc_vector).float()
        inputs = torch.cat([origin_vector, entity_desc_vector], dim=1).permute(1, 0).unsqueeze(0) # bsz x len x 300
        inputs = inputs.to(device)
        preds = model(inputs)
        score = preds.view(-1).tolist()[1]
#         print(preds)
        scores.append(score)
    scores = torch.tensor(scores)
    index = scores.argmax()
    return entity_descs[index]


In [35]:
model = RNNModel(embedding_size, hidden_size, num_layers, num_directions, num_classes)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.BCELoss()
model, total_train_loss, mean_loss = train(model, train_set, test_set, optimizer, loss_func, epochs, model_save_path='model/word_sense_disambiguation.bin')


Epoch 0/ 10 train_loss  0.7609254479408264
Epoch 0/ 20 train_loss  0.7499568283557891
Epoch 0/ 30 train_loss  0.7316730082035064
Epoch 0/ 40 train_loss  0.7303471058607102
Epoch 0/ 50 train_loss  0.7237918889522552
Epoch 0/ 60 train_loss  0.7180019507805506
Epoch 0/ 70 train_loss  0.7168742792946952
Epoch 0/ 80 train_loss  0.7173353753983974
Epoch 0/ 90 train_loss  0.7143663909700182
Epoch 0/ 100 train_loss  0.7075330072641373
Epoch 0/ 110 train_loss  0.7070197755640203
Epoch 0/ 120 train_loss  0.7114212090770403
Epoch 0/ 130 train_loss  0.7098383688009702
Epoch 0/ 140 train_loss  0.7037482129676002
Epoch 0/ 150 train_loss  0.7035306203365326
Epoch 0/ 160 train_loss  0.7040739387273789
Epoch 0/ 170 train_loss  0.7044844048864701
Epoch 0/ 180 train_loss  0.7009855667750041
Epoch 0/ 190 train_loss  0.7045519050798918
Epoch 0/ 200 train_loss  0.7066385893523693
Epoch 0/ 210 train_loss  0.7044750371149608
Epoch 0/ 220 train_loss  0.7037273194302213
Epoch 0/ 230 train_loss  0.70181181689967

Epoch 0/ 1870 train_loss  0.664039981203841
Epoch 0/ 1880 train_loss  0.6636209924586435
Epoch 0/ 1890 train_loss  0.6629181957059594
Epoch 0/ 1900 train_loss  0.6627698415968764
Epoch 0/ 1910 train_loss  0.6630606487334867
Epoch 0/ 1920 train_loss  0.6616936132331224
Epoch 0/ 1930 train_loss  0.6617556044802919
Epoch 0/ 1940 train_loss  0.660892544615745
Epoch 0/ 1950 train_loss  0.6603791097169504
Epoch 0/ 1960 train_loss  0.6589679203300299
Epoch 0/ 1970 train_loss  0.6584047248426277
Epoch 0/ 1980 train_loss  0.658219849805564
Epoch 0/ 1990 train_loss  0.6583045758325701
Epoch 0/ 2000 train_loss  0.6584837984871119
Epoch 0/ 2010 train_loss  0.6575571612728324
Epoch 0/ 2020 train_loss  0.6569949178430852
Epoch 0/ 2030 train_loss  0.6558462835462957
Epoch 0/ 2040 train_loss  0.6552847784761267
Epoch 0/ 2050 train_loss  0.654744112849599
Epoch 0/ 2060 train_loss  0.6538363493599214
Epoch 0/ 2070 train_loss  0.6529251705891125
Epoch 0/ 2080 train_loss  0.6512663467017074
Epoch 0/ 2090 

Epoch 0/ 3700 train_loss  0.6309560789273598
Train Loss: 0.6311649910361984, Train Acc: 0.6458670988654781
Test Loss: 0.5892059767449443, Test Acc: 0.7012987012987013
保存模型成功! 路径为: ’model/word_sense_disambiguation.bin‘
Epoch 1/ 3710 train_loss  0.0012216358796324751
Epoch 1/ 3720 train_loss  0.003153974120435093
Epoch 1/ 3730 train_loss  0.004975258854794637
Epoch 1/ 3740 train_loss  0.006527682607126943
Epoch 1/ 3750 train_loss  0.008329082848807214
Epoch 1/ 3760 train_loss  0.00980991002863792
Epoch 1/ 3770 train_loss  0.011340435377107255
Epoch 1/ 3780 train_loss  0.013456869647872707
Epoch 1/ 3790 train_loss  0.014707755159989705
Epoch 1/ 3800 train_loss  0.016032492826428105
Epoch 1/ 3810 train_loss  0.017250208863846782
Epoch 1/ 3820 train_loss  0.01835474719296345
Epoch 1/ 3830 train_loss  0.020077475253910876
Epoch 1/ 3840 train_loss  0.021205495058461907
Epoch 1/ 3850 train_loss  0.022185186024977828
Epoch 1/ 3860 train_loss  0.02378269226785386
Epoch 1/ 3870 train_loss  0.0251

Epoch 1/ 5470 train_loss  0.16592355909628262
Epoch 1/ 5480 train_loss  0.16601314850038015
Epoch 1/ 5490 train_loss  0.1661102887154345
Epoch 1/ 5500 train_loss  0.16604951882560826
Epoch 1/ 5510 train_loss  0.1676156196357402
Epoch 1/ 5520 train_loss  0.1682502151456311
Epoch 1/ 5530 train_loss  0.1689096730953086
Epoch 1/ 5540 train_loss  0.16944393456004403
Epoch 1/ 5550 train_loss  0.1704832966108379
Epoch 1/ 5560 train_loss  0.17135404372288307
Epoch 1/ 5570 train_loss  0.17147781304923462
Epoch 1/ 5580 train_loss  0.1717384247239012
Epoch 1/ 5590 train_loss  0.17248142716671763
Epoch 1/ 5600 train_loss  0.17295512744087863
Epoch 1/ 5610 train_loss  0.1736820905966927
Epoch 1/ 5620 train_loss  0.17428361927055064
Epoch 1/ 5630 train_loss  0.17490612090929525
Epoch 1/ 5640 train_loss  0.17532174992089739
Epoch 1/ 5650 train_loss  0.1760890125293252
Epoch 1/ 5660 train_loss  0.17608345071945092
Epoch 1/ 5670 train_loss  0.17662645262797783
Epoch 1/ 5680 train_loss  0.17698645109853

Epoch 1/ 7290 train_loss  0.2364444716932822
Epoch 1/ 7300 train_loss  0.23649718051198618
Epoch 1/ 7310 train_loss  0.23680223693670108
Epoch 1/ 7320 train_loss  0.23693162676546461
Epoch 1/ 7330 train_loss  0.23713751008329054
Epoch 1/ 7340 train_loss  0.23740956616173287
Epoch 1/ 7350 train_loss  0.23790080715360085
Epoch 1/ 7360 train_loss  0.2381592187985839
Epoch 1/ 7370 train_loss  0.2383608029678545
Epoch 1/ 7380 train_loss  0.2387759972567794
Epoch 1/ 7390 train_loss  0.238966502638874
Epoch 1/ 7400 train_loss  0.239200404198503
Train Loss: 0.4784020120376114, Train Acc: 0.7809292274446246
Test Loss: 0.4600255056446896, Test Acc: 0.7792207792207793
保存模型成功! 路径为: ’model/word_sense_disambiguation.bin‘
Epoch 2/ 7410 train_loss  0.0002683803073411497
Epoch 2/ 7420 train_loss  0.0010571555494321453
Epoch 2/ 7430 train_loss  0.002133090404010273
Epoch 2/ 7440 train_loss  0.00263792844639562
Epoch 2/ 7450 train_loss  0.003197120435837248
Epoch 2/ 7460 train_loss  0.003843172429125803


...

Epoch 3/ 14260 train_loss  0.06473917470152933
Epoch 3/ 14270 train_loss  0.064799523673695
Epoch 3/ 14280 train_loss  0.06485257454480929
Epoch 3/ 14290 train_loss  0.06502602239294802
Epoch 3/ 14300 train_loss  0.06502513885525388
Epoch 3/ 14310 train_loss  0.06503831837405676
Epoch 3/ 14320 train_loss  0.06503952452338642
Epoch 3/ 14330 train_loss  0.06549197015141313
Epoch 3/ 14340 train_loss  0.06549919803498717
Epoch 3/ 14350 train_loss  0.06548118116107605
Epoch 3/ 14360 train_loss  0.06547119897388684
Epoch 3/ 14370 train_loss  0.06564819132501247
Epoch 3/ 14380 train_loss  0.06571109310950243
Epoch 3/ 14390 train_loss  0.06582539149144902
Epoch 3/ 14400 train_loss  0.06593416793746824
Epoch 3/ 14410 train_loss  0.06599959712113171
Epoch 3/ 14420 train_loss  0.06598223710105086
Epoch 3/ 14430 train_loss  0.0660889229345134
Epoch 3/ 14440 train_loss  0.06605297086829177
Epoch 3/ 14450 train_loss  0.06617507650197256
Epoch 3/ 14460 train_loss  0.0663802036310561
Epoch 3/ 14470 tr

Epoch 4/ 15960 train_loss  0.019150287226523927
Epoch 4/ 15970 train_loss  0.019256397486569907
Epoch 4/ 15980 train_loss  0.019444846251043514
Epoch 4/ 15990 train_loss  0.01953378870833723
Epoch 4/ 16000 train_loss  0.019717325680424648
Epoch 4/ 16010 train_loss  0.019821212091367953
Epoch 4/ 16020 train_loss  0.019911400591364183
Epoch 4/ 16030 train_loss  0.02016854393192204
Epoch 4/ 16040 train_loss  0.020220591860317563
Epoch 4/ 16050 train_loss  0.020465331687302954
Epoch 4/ 16060 train_loss  0.02061808841896922
Epoch 4/ 16070 train_loss  0.020778599587127095
Epoch 4/ 16080 train_loss  0.020844449003597252
Epoch 4/ 16090 train_loss  0.020981045190842644
Epoch 4/ 16100 train_loss  0.021005539579078747
Epoch 4/ 16110 train_loss  0.02112245657587952
Epoch 4/ 16120 train_loss  0.02139873410662659
Epoch 4/ 16130 train_loss  0.021548926418288712
Epoch 4/ 16140 train_loss  0.021596000398967752
Epoch 4/ 16150 train_loss  0.021697237237899063
Epoch 4/ 16160 train_loss  0.0219501225911197

Epoch 4/ 17690 train_loss  0.038849153750179
Epoch 4/ 17700 train_loss  0.03887276160492166
Epoch 4/ 17710 train_loss  0.03893338749758632
Epoch 4/ 17720 train_loss  0.03912026182310796
Epoch 4/ 17730 train_loss  0.03921682987166966
Epoch 4/ 17740 train_loss  0.039268770450287406
Epoch 4/ 17750 train_loss  0.03937315685247315
Epoch 4/ 17760 train_loss  0.03951573155802142
Epoch 4/ 17770 train_loss  0.039536723951580026
Epoch 4/ 17780 train_loss  0.03962976607854952
Epoch 4/ 17790 train_loss  0.039778444401698865
Epoch 4/ 17800 train_loss  0.039892620564955024
Epoch 4/ 17810 train_loss  0.0399444808353347
Epoch 4/ 17820 train_loss  0.039924409118899586
Epoch 4/ 17830 train_loss  0.04019487650401573
Epoch 4/ 17840 train_loss  0.040272714112961884
Epoch 4/ 17850 train_loss  0.040265133801479974
Epoch 4/ 17860 train_loss  0.04032443531535859
Epoch 4/ 17870 train_loss  0.040403763102665996
Epoch 4/ 17880 train_loss  0.04056359659416963
Epoch 4/ 17890 train_loss  0.04055965782434115
Epoch 4/

Epoch 5/ 19410 train_loss  0.012030769520436128
Epoch 5/ 19420 train_loss  0.012073892461153767
Epoch 5/ 19430 train_loss  0.01245643868674521
Epoch 5/ 19440 train_loss  0.012572005339632321
Epoch 5/ 19450 train_loss  0.012683113176809008
Epoch 5/ 19460 train_loss  0.012871315747076917
Epoch 5/ 19470 train_loss  0.012927968305293854
Epoch 5/ 19480 train_loss  0.013012952434480982
Epoch 5/ 19490 train_loss  0.013091631387972376
Epoch 5/ 19500 train_loss  0.013164059879420998
Epoch 5/ 19510 train_loss  0.013218863116653268
Epoch 5/ 19520 train_loss  0.013221531256268041
Epoch 5/ 19530 train_loss  0.013234728458336037
Epoch 5/ 19540 train_loss  0.013476768602923907
Epoch 5/ 19550 train_loss  0.013622458083196742
Epoch 5/ 19560 train_loss  0.013677572917473853
Epoch 5/ 19570 train_loss  0.013908466566700363
Epoch 5/ 19580 train_loss  0.013943380416335652
Epoch 5/ 19590 train_loss  0.014003609460663882
Epoch 5/ 19600 train_loss  0.014001829665405502
Epoch 5/ 19610 train_loss  0.014013595247

Epoch 5/ 21150 train_loss  0.026952668919406977
Epoch 5/ 21160 train_loss  0.026955096579077197
Epoch 5/ 21170 train_loss  0.026951480210698136
Epoch 5/ 21180 train_loss  0.027096040014387625
Epoch 5/ 21190 train_loss  0.02714825968048155
Epoch 5/ 21200 train_loss  0.02718438727416576
Epoch 5/ 21210 train_loss  0.027223588181255545
Epoch 5/ 21220 train_loss  0.027473845360016045
Epoch 5/ 21230 train_loss  0.027527400536998106
Epoch 5/ 21240 train_loss  0.02765974189615834
Epoch 5/ 21250 train_loss  0.02769242186367923
Epoch 5/ 21260 train_loss  0.02773223536835386
Epoch 5/ 21270 train_loss  0.027918230913069966
Epoch 5/ 21280 train_loss  0.027918147983501562
Epoch 5/ 21290 train_loss  0.02799099098936412
Epoch 5/ 21300 train_loss  0.028062368663053165
Epoch 5/ 21310 train_loss  0.028065275783886554
Epoch 5/ 21320 train_loss  0.028084596590129882
Epoch 5/ 21330 train_loss  0.02809923195797922
Epoch 5/ 21340 train_loss  0.0281025969931921
Epoch 5/ 21350 train_loss  0.028111687007439806
E

In [372]:
origin = '苹果12马上就要发布了。'
entity_descs = ['《苹果》是由李玉执导，范冰冰、佟大为、梁家辉、金燕玲领衔主演的黑色幽默剧情电影。于2007年5月18日在中国大陆上映。影片讲述了两个贫富家庭之间离奇诡异的矛盾冲突和感情错位的戏剧性故事。'
, 'iPhone 是美国苹果公司研发的iPhone手机，采用了直面边框设计，支持5G，搭载A14 Bionic芯片，双镜头后置摄像头系统。支持北斗导航 [1]  ，有黑色、白色、红色、绿色、蓝色五种配色。'
]

origin = '我们家的苹果快要熟了，你要不要来几斤？'
entity_descs = ['《苹果》是由李玉执导，范冰冰、佟大为、梁家辉、金燕玲领衔主演的黑色幽默剧情电影。于2007年5月18日在中国大陆上映。影片讲述了两个贫富家庭之间离奇诡异的矛盾冲突和感情错位的戏剧性故事。'
, 'iPhone 是美国苹果公司研发的iPhone手机，采用了直面边框设计，支持5G，搭载A14 Bionic芯片，双镜头后置摄像头系统。支持北斗导航 [1]  ，有黑色、白色、红色、绿色、蓝色五种配色。',
 '苹果，又称柰或林檎，是苹果 树（学名：Malus domestica）的果实。苹果树是蔷薇科苹果亚科苹果属植物，为落叶乔木，果实富含矿物质和维生素，是人们最常食用的水果之一。'                    
]
right_text = predict(model, origin, entity_descs)
print(right_text)

苹果，又称柰或林檎，是苹果树（学名：Malus domestica）的果实。苹果树是蔷薇科苹果亚科苹果属植物，为落叶乔木，果实富含矿物质和维生素，是人们最常食用的水果之一。
